# Heating oil COT MM Nowcasting EDA 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")


In [3]:
import numpy as np
import pandas as pd 

In [4]:
import scipy.stats as stats

In [5]:
import sys
sys.path.append('../../../')

In [6]:
import matplotlib.pyplot as plt 

In [7]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


In [8]:
from src.utils.dates import get_nyse_business_dates
from src.utils.io.read import PreprocessedDataReader

In [9]:
from src.preprocessing.base import FutureTicker

In [10]:
from src.settings import Settings

In [11]:
from research.metrics import get_features_responses_correlation
from research.plots import plot_bin_summary_of_xy

In [12]:
pdr = PreprocessedDataReader(Settings.historical.paths.PREPROCESSED_DATA_PATH)                    

In [13]:
dataset = pdr.read_dataset(ticker=FutureTicker.HEATING_OIL)

In [14]:
dataset.shape

(809, 81)

In [15]:
dataset.tail(3)

,tradeDate,Name,Commercial_NetPosition,CommercialLongPosition,CommercialShortPosition,ManagedMoney_NetPosition,ManagedMoney_LongPosition,ManagedMoney_ShortPosition,Commercial_NetPosition_change,prior_report_Commercial_NetPosition_change,...,prior_cumulative_5D_F2_Volume_change,next_prior_cumulative_5D_F2_Volume_change,prior_cumulative_5D_F1MinusF2_Volume_change,next_prior_cumulative_5D_F1MinusF2_Volume_change,F1_RolledPrice_change,next_F1_RolledPrice_change,F2_RolledPrice_change,next_F2_RolledPrice_change,F3_RolledPrice_change,next_F3_RolledPrice_change
806,2025-06-17,HO,-94113.0,72420.0,166533.0,2156.0,43088.0,40932.0,-1096.0,-15055.0,...,250896.0,116650.0,-178799.0,-247835.0,36.35,-22.00,34.40,-22.15,32.91,-22.31
807,2025-06-24,HO,-101062.0,60660.0,161722.0,10738.0,52627.0,41889.0,-6949.0,-1096.0,...,116650.0,-221825.0,-247835.0,104997.0,-22.00,10.81,-22.15,5.59,-22.31,4.55
808,2025-07-01,HO,-104727.0,71449.0,176176.0,12898.0,54458.0,41560.0,-3665.0,-6949.0,...,-221825.0,NaN,104997.0,NaN,10.81,NaN,5.59,NaN,4.55,NaN


In [16]:
dataset['tradeDate'].min(),dataset['tradeDate'].max()

('2010-01-05', '2025-07-01')

In [17]:
dataset['tradeDate'] = pd.to_datetime(dataset['tradeDate']).dt.date 

In [18]:
forward_response_names = ['forward_ManagedMoney_NetPosition_to_openinterest_change',
                            'forward_ManagedMoney_LongPosition_to_openinterest_change',
                            'forward_ManagedMoney_ShortPosition_to_openinterest_change']

prior_response_names = ['prior_report_ManagedMoney_NetPosition_to_openinterest_change',
                            'prior_report_ManagedMoney_LongPosition_to_openinterest_change',
                            'prior_report_ManagedMoney_ShortPosition_to_openinterest_change']


In [19]:
get_features_responses_correlation(dataset = dataset,
                                   features = prior_response_names,
                                   responses=forward_response_names).sort_values(by = forward_response_names[0])

,Feature,forward_ManagedMoney_NetPosition_to_openinterest_change,forward_ManagedMoney_LongPosition_to_openinterest_change,forward_ManagedMoney_ShortPosition_to_openinterest_change
1,prior_report_ManagedMoney_LongPosition_to_open...,-0.080357,-0.121587,-0.056700
0,prior_report_ManagedMoney_NetPosition_to_openi...,-0.068686,-0.100237,-0.032330
2,prior_report_ManagedMoney_ShortPosition_to_ope...,0.000089,-0.015450,-0.026969
